In [1]:
import os
import pandas as pd
import numpy as np
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import copy
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import joblib
import optuna
from tqdm import tqdm
import time
import codecs
import pickle

# Configuration

In [2]:
rawdata_dir = 'rawdata'
data_dir = 'data'

lags = [1,2,3,6,12,24,36]

# NUM_FOLD_DAYS = 28
NUM_FOLD_DAYS = 56
VALID_FIRST_DAY = 1914
VALID_LAST_DAY = 1941
NUM_USE_DAY = 365
# TRAIN_FIRST_DAY = max(lags)
TRAIN_FIRST_DAY = VALID_FIRST_DAY - NUM_USE_DAY + 1

OPTUNA_TRIAL = 70

# Read data

In [3]:
sales = pd.read_csv(os.path.join(rawdata_dir, 'sales_train_evaluation.csv'))
sales.name = 'sales'
calendar = pd.read_csv(os.path.join(rawdata_dir, 'calendar.csv'))
calendar.name = 'calendar'
prices = pd.read_csv(os.path.join(rawdata_dir, 'sell_prices.csv'))
prices.name = 'prices'

Since, the validation data is now available for the days 1914-1941, Adding zero sales for days: d_1942 - d_1969(Test)

In [4]:
#Add zero sales for the remaining days 1942-1969
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)

# Downcast

In [5]:
#Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

# Melt data

In [6]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

## Combine data
Combine price data from prices dataframe and days data from calendar dataset.

In [7]:
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left')

# Feature Engineering

## Label Encoding

In [8]:
#Store the categories along with their codes
d_id = dict(zip(df.id.cat.codes, df.id))
d_item_id = dict(zip(df.item_id.cat.codes, df.item_id))
d_dept_id = dict(zip(df.dept_id.cat.codes, df.dept_id))
d_cat_id = dict(zip(df.cat_id.cat.codes, df.cat_id))
d_store_id = dict(zip(df.store_id.cat.codes, df.store_id))
d_state_id = dict(zip(df.state_id.cat.codes, df.state_id))

In [9]:
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
cols = df.dtypes.index.tolist()
types = df.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category':
        df[cols[i]] = df[cols[i]].cat.codes
        
df.drop('date',axis=1,inplace=True)

## Introduce lags

In [10]:
#Introduce lags
def make_lags(df, lags):
    for lag in lags:
        df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)
        
    return df

In [11]:
df = make_lags(df, lags)

## make new features

In [12]:
def make_new_feature(df, d_last = df['d'].max()):
    # copy sold
    sold_tmp = copy.copy(df['sold'])
    
    # embed NaN
    df['sold'][df['d'] > d_last] = np.nan
    
    # make feature using sold
    df['item_sold_avg'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)
    df['state_sold_avg'] = df.groupby('state_id')['sold'].transform('mean').astype(np.float16)
    df['store_sold_avg'] = df.groupby('store_id')['sold'].transform('mean').astype(np.float16)
    df['cat_sold_avg'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
    df['dept_sold_avg'] = df.groupby('dept_id')['sold'].transform('mean').astype(np.float16)
    df['cat_dept_sold_avg'] = df.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
    df['store_item_sold_avg'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['cat_item_sold_avg'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['dept_item_sold_avg'] = df.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['state_store_sold_avg'] = df.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
    df['state_store_cat_sold_avg'] = df.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
    df['store_cat_dept_sold_avg'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
    
    df['rolling_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
    df['expanding_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.expanding(2).mean()).astype(np.float16)
    
    df['daily_avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','d'])['sold'].transform('mean').astype(np.float16)
    df['avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform('mean').astype(np.float16)
    df['selling_trend'] = (df['daily_avg_sold'] - df['avg_sold']).astype(np.float16)
    df.drop(['daily_avg_sold','avg_sold'],axis=1,inplace=True)
    
    # restore sold
    df['sold'] = sold_tmp
    
    return df

In [13]:
df_all = copy.copy(df)
df_all = make_new_feature(df_all)
df_all = df_all[df_all['d'] >= max(lags)]
df_all.info()

df_all.to_pickle(os.path.join(data_dir, 'data_all.pkl'))   # save df_all (use this after CV)
del df_all
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58967660 entries, 1067150 to 60034809
Data columns (total 43 columns):
id                          int16
item_id                     int16
dept_id                     int8
cat_id                      int8
store_id                    int8
state_id                    int8
d                           int16
sold                        int16
wm_yr_wk                    int16
weekday                     int8
wday                        int8
month                       int8
year                        int16
event_name_1                int8
event_type_1                int8
event_name_2                int8
event_type_2                int8
snap_CA                     int8
snap_TX                     int8
snap_WI                     int8
sell_price                  float16
sold_lag_1                  float16
sold_lag_2                  float16
sold_lag_3                  float16
sold_lag_6                  float16
sold_lag_12                 float

0

# Cross Validation

## Cross Validation for each store (function)

In [14]:
def xgboost_cv_store(df, store):
    def objective(trial):
        # param space
        param = {
            'objective': 'reg:squarederror',
            'n_estimators': 1000,
            'learning_rate': 0.1,
            'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 0.95, 0.05),
            'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.6, 0.95, 0.05),
            'max_depth':  trial.suggest_int('max_depth', 3, 9),
            'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
            'min_child_weight': trial.suggest_loguniform('min_child_weight', 0.1, 10.0),
            'tree_method': 'gpu_hist'
        }
        
        ### START: CV Loop ###
        isBreakCV = False
        fold_iter = 0
        rmse_mean = 0.
        while not(isBreakCV):
            start = time.time()   # start timer
            
            d_valid_last = VALID_LAST_DAY - NUM_FOLD_DAYS * fold_iter
            d_train_last = VALID_LAST_DAY - NUM_FOLD_DAYS * (fold_iter + 1)
            if d_train_last < TRAIN_FIRST_DAY + NUM_FOLD_DAYS * 2:
                isBreakCV = True
            
            # make_new_feature
            df_fold_file = os.path.join(data_dir, 'data_foldday'+str(NUM_FOLD_DAYS)+'_fold'+str(fold_iter + 1)+'.pkl')
            if os.path.isfile(df_fold_file):   # make_new_feature 適用後のファイルがあればそれを使う
                df_fold = pd.read_pickle(df_fold_file)
            else:
                df_fold = df[df['d'] <= d_valid_last]
                df_fold = make_new_feature(df_fold, d_train_last)   # train の情報だけを使って特徴量を作成
                df_fold.to_pickle(df_fold_file)   # pickleとして保存しておく

            # Extract store
            df_fold = df_fold[df_fold['store_id'] == store]
                        
            # Split data
            X_train = df_fold[(df_fold['d'] >= TRAIN_FIRST_DAY) & (df_fold['d'] <= d_train_last)].drop('sold', axis=1)
            y_train = df_fold[(df_fold['d'] >= TRAIN_FIRST_DAY) & (df_fold['d'] <= d_train_last)]['sold']
            X_valid = df_fold[(df_fold['d'] > d_train_last) & (df_fold['d'] <= d_valid_last)].drop('sold', axis=1)
            y_valid = df_fold[(df_fold['d'] > d_train_last) & (df_fold['d'] <= d_valid_last)]['sold']
            
            # Train XGBoost model
            model = xgb.XGBRegressor(**param)
#             model.fit(X_train, y_train, eval_set=[(X_train,y_train), (X_valid, y_valid)],
#                  eval_metric='rmse', verbose=10, early_stopping_rounds=100)
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
                  eval_metric='rmse', verbose=10, early_stopping_rounds=50)
            y_pred = model.predict(X_valid)
            rmse_mean += np.sqrt(mean_squared_error(y_valid.values, y_pred))
            
            fold_iter += 1
            elapsed_time = time.time() - start   # stop timer
            print("[{0}] Fold{1}: {2} [sec]".format(d_store_id[store], fold_iter, elapsed_time))
        ### END: CV Loop ###
        
        del df_fold, model, X_train, y_train, X_valid, y_valid, y_pred
        gc.collect()
        
        rmse_mean /= fold_iter
        return rmse_mean
    return objective


## Execute CV

In [17]:
stores = sales.store_id.cat.codes.unique().tolist()

valid_preds = df[(df['d'] >= VALID_FIRST_DAY) & (df['d'] <= VALID_LAST_DAY)]['sold']
eval_preds = df[df['d'] > VALID_LAST_DAY]['sold']

# find best params by doing CV (optuna)
for store in tqdm(stores[1:5]):
    print('##################### CV START: {0} #####################'.format(d_store_id[store]))
    study = optuna.create_study()
    study.optimize(xgboost_cv_store(df, store), n_trials=OPTUNA_TRIAL)
    
    # save optuna log
    df_trial = study.trials_dataframe()
    df_trial.to_pickle(os.path.join('report', 'OptunaTrials_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'))
    del df_trial
    gc.collect()
    
    # savae best params found by CV
    bestparam_filename = 'BestParamsSelectedByCV_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'wb') as f:
        pickle.dump(study.best_params, f)



  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

##################### CV START: CA_2 #####################
[0]	validation_0-rmse:3.18332
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.77326
[20]	validation_0-rmse:2.56906
[30]	validation_0-rmse:2.53782
[40]	validation_0-rmse:2.46784
[50]	validation_0-rmse:2.47968
[60]	validation_0-rmse:2.55044
[70]	validation_0-rmse:2.55752
[80]	validation_0-rmse:2.62764
[90]	validation_0-rmse:2.64106
[100]	validation_0-rmse:2.83880
Stopping. Best iteration:
[53]	validation_0-rmse:2.45774

[CA_2] Fold1: 9.677758932113647 [sec]
[0]	validation_0-rmse:3.06903
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.57485
[20]	validation_0-rmse:2.46110
[30]	validation_0-rmse:2.43606
[40]	validation_0-rmse:2.42880
[50]	validation_0-rmse:2.43282
[60]	validation_0-rmse:2.50614
[70]	validation_0-rmse:2.55850
[80]	validation_0-rmse:2.56678
[90]	validation_0-rmse:2.57905
[100]	validation_0-rmse:2.58398
Stopping. Best iteration:
[53]	va

[I 2020-06-25 11:36:18,882] Finished trial#0 with value: 2.503983163833618 with parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'max_depth': 5, 'gamma': 0.0017222945942962906, 'min_child_weight': 0.224450074454338}. Best is trial#0 with value: 2.503983163833618.


[0]	validation_0-rmse:3.17775
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.62902
[20]	validation_0-rmse:2.54524
[30]	validation_0-rmse:2.55235
[40]	validation_0-rmse:2.62976
[50]	validation_0-rmse:2.63649
[60]	validation_0-rmse:2.64143
Stopping. Best iteration:
[18]	validation_0-rmse:2.54330

[CA_2] Fold1: 10.765116930007935 [sec]
[0]	validation_0-rmse:3.05995
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.50719
[20]	validation_0-rmse:2.44875
[30]	validation_0-rmse:2.46402
[40]	validation_0-rmse:2.49791
[50]	validation_0-rmse:2.49929
[60]	validation_0-rmse:2.51335
Stopping. Best iteration:
[18]	validation_0-rmse:2.44585

[CA_2] Fold2: 10.10924744606018 [sec]
[0]	validation_0-rmse:3.01690
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.57477
[20]	validation_0-rmse:2.49168
[30]	validation_0-rmse:2.50565
[40]	validation_0-rmse:2.55533
[50]	validation_0-rmse:2.5

[I 2020-06-25 11:37:05,646] Finished trial#1 with value: 2.54998574256897 with parameters: {'subsample': 0.65, 'colsample_bytree': 0.8, 'max_depth': 8, 'gamma': 4.6493563682958145e-07, 'min_child_weight': 3.145261073332095}. Best is trial#0 with value: 2.503983163833618.


...

[I 2020-06-25 14:53:27,151] Finished trial#67 with value: 2.2229659080505373 with parameters: {'subsample': 0.95, 'colsample_bytree': 0.6, 'max_depth': 4, 'gamma': 1.5618817927607602e-08, 'min_child_weight': 0.11918034589035374}. Best is trial#51 with value: 2.1383450031280518.


[0]	validation_0-rmse:3.02315
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.54403
[20]	validation_0-rmse:2.36741
[30]	validation_0-rmse:2.29747
[40]	validation_0-rmse:2.29466
[50]	validation_0-rmse:2.28900
[60]	validation_0-rmse:2.28134
[70]	validation_0-rmse:2.25875
[80]	validation_0-rmse:2.28835
[90]	validation_0-rmse:2.30114
[100]	validation_0-rmse:2.30166
[110]	validation_0-rmse:2.31193
[120]	validation_0-rmse:2.37870
Stopping. Best iteration:
[71]	validation_0-rmse:2.25744

[TX_1] Fold1: 9.877125263214111 [sec]
[0]	validation_0-rmse:2.94018
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.47578
[20]	validation_0-rmse:2.29232
[30]	validation_0-rmse:2.21991
[40]	validation_0-rmse:2.18447
[50]	validation_0-rmse:2.16207
[60]	validation_0-rmse:2.12363
[70]	validation_0-rmse:2.12104
[80]	validation_0-rmse:2.12067
[90]	validation_0-rmse:2.12275
[100]	validation_0-rmse:2.11574
[110]	validation_0-rmse:2.11

[I 2020-06-25 14:54:08,841] Finished trial#68 with value: 2.1541256427764894 with parameters: {'subsample': 0.75, 'colsample_bytree': 0.65, 'max_depth': 5, 'gamma': 1.0590297614257536e-08, 'min_child_weight': 0.219254890275378}. Best is trial#51 with value: 2.1383450031280518.


[0]	validation_0-rmse:3.07718
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.58872
[20]	validation_0-rmse:2.34408
[30]	validation_0-rmse:2.28754
[40]	validation_0-rmse:2.27092
[50]	validation_0-rmse:2.26894
[60]	validation_0-rmse:2.26176
[70]	validation_0-rmse:2.26437
[80]	validation_0-rmse:2.33802
[90]	validation_0-rmse:2.35645
[100]	validation_0-rmse:2.33705
Stopping. Best iteration:
[54]	validation_0-rmse:2.23079

[TX_1] Fold1: 9.479262590408325 [sec]
[0]	validation_0-rmse:2.99145
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.52536
[20]	validation_0-rmse:2.36561
[30]	validation_0-rmse:2.29822
[40]	validation_0-rmse:2.27817
[50]	validation_0-rmse:2.27681
[60]	validation_0-rmse:2.30582
[70]	validation_0-rmse:2.52760
[80]	validation_0-rmse:2.54354
[90]	validation_0-rmse:2.54480
[100]	validation_0-rmse:2.54818
Stopping. Best iteration:
[51]	validation_0-rmse:2.27343

[TX_1] Fold2: 8.983668565750122 [s

[I 2020-06-25 14:54:49,145] Finished trial#69 with value: 2.228553533554077 with parameters: {'subsample': 0.8, 'colsample_bytree': 0.7, 'max_depth': 5, 'gamma': 1.4193088647733672e-07, 'min_child_weight': 0.5522130555964279}. Best is trial#51 with value: 2.1383450031280518.

100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [3:19:11<00:00, 2987.76s/it]


In [ ]:
del df
gc.collect()

## Train & Predict by best model (function)

In [ ]:
def xgboost_by_bestparam_for_valid(df, store, best_params, valid_preds):
    # prepare data set
    df_store = df[df['store_id'] == store]
    X_train = df_store[df_store['d'] < VALID_FIRST_DAY].drop('sold', axis=1)
    y_train = df_store[df_store['d'] < VALID_FIRST_DAY]['sold']
    X_valid = df_store[(df_store['d'] >= VALID_FIRST_DAY) & (df_store['d'] <= VALID_LAST_DAY)].drop('sold', axis=1)
    
    # train XGBoost by using best_params
    model = xgb.XGBRegressor(**best_params)
    model.fit(X_train, y_train, eval_metric='rmse')
    
    # save model for validation
    modelfile = 'BestModel4valid_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    with open(os.path.join('result', 'best_model', modelfile), 'wb') as f:
        pickle.dump(model, f)
    
    # predict for valid
    valid_preds[X_valid.index] = model.predict(X_valid)
    del df_store, X_train, y_train, X_valid, model
    gc.collect()
    

In [ ]:
def xgboost_by_bestparam_for_test(df, store, best_params, eval_preds):
    # prepare data set
    df_store = df[df['store_id'] == store]
    X_train = df_store[df_store['d'] <= VALID_LAST_DAY].drop('sold', axis=1)
    y_train = df_store[df_store['d'] <= VALID_LAST_DAY]['sold']
    X_test = df_store[df_store['d'] > VALID_LAST_DAY].drop('sold', axis=1)
    
    # train XGBoost by using best_params
    model = xgb.XGBRegressor(**best_params)
    model.fit(X_train, y_train, eval_metric='rmse')
        
    # predict for test
    eval_preds[X_test.index] = model.predict(X_test)
    
    # save model for test
    modelfile = 'BestModel4test_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    with open(os.path.join('result', 'best_model', modelfile), 'wb') as f:
        pickle.dump(model, f)
    
    del df_store, X_train, y_train, X_test, model
    gc.collect()
    

## Train & Predict by best model

In [ ]:
# train & predict (best model)
df_all = pd.read_pickle(os.path.join(data_dir, 'data_all.pkl'))
for store in tqdm(stores):
    # best model for valid
    xgboost_by_bestparam_for_valid(df_all, store, study.best_params, valid_preds)
    
    # best model for test
    xgboost_by_bestparam_for_test(df_all, store, study.best_params, eval_preds)


In [ ]:
del df_all
gc.collect()

# Make submission

In [ ]:
#Set actual equal to false if you want to top in the public leaderboard :P
actual = True
if actual == False:
    #Get the validation results(We already have them as less than one month left for competition to end)
    validation = sales[['id']+['d_' + str(i) for i in range(1914,1942)]]
    validation['id']=pd.read_csv(os.path.join(rawdata_dir, 'sales_train_validation.csv')).id
    validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
else:
    #Get the actual validation results
    valid['sold'] = valid_preds
    validation = valid[['id','d','sold']]
    validation = pd.pivot(validation, index='id', columns='d', values='sold').reset_index()
    validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
    validation.id = validation.id.map(d_id).str.replace('evaluation','validation')

#Get the evaluation results
test['sold'] = eval_preds
evaluation = test[['id','d','sold']]
evaluation = pd.pivot(evaluation, index='id', columns='d', values='sold').reset_index()
evaluation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
#Remap the category id to their respective categories
evaluation.id = evaluation.id.map(d_id)

#Prepare the submission
submit = pd.concat([validation,evaluation]).reset_index(drop=True)
submit_file = 'xgboost_cv_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.csv'
submit.to_csv(os.path.join('submission_point', submit_file), index=False)